Imports

In [98]:
import cv2
import numpy as np
import os

In [99]:
show_steps = False

Load Image

In [100]:
file_dir = "data"
file_name = "PennAir 2024 App Static.png"
file = os.path.join(file_dir, file_name)

In [101]:
# Load image
img = cv2.imread(file)

In [102]:
img = cv2.resize(img, (1920, 1080))

In [103]:
if show_steps:
    # Display original image
    cv2.imshow("Original", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

Binary Value (Brightness)

In [104]:
# Convert to Hue Saturation Value format
hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

In [105]:
# Extract only Value (Brightness)
val_img = hsv_img[:,:,2]

# Set brightness threshold x
x = 140
_, bin_val_img = cv2.threshold(val_img, x, 255, cv2.THRESH_BINARY)

In [106]:
if show_steps:
    # Display binary value image
    cv2.imshow("Binary Value (Brightness)", bin_val_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

In [107]:
# Clean up w/ morphological operations
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))

# Remove noise
smoothed_img = cv2.morphologyEx(bin_val_img, cv2.MORPH_OPEN, kernel)

# Fill gaps + smooth edges
smoothed_img = cv2.morphologyEx(smoothed_img, cv2.MORPH_CLOSE, kernel)

# Smooth edges again
smoothed_img = cv2.medianBlur(smoothed_img, 5)

In [108]:
if show_steps:
    # Display smoothed binary value image
    cv2.imshow("Smoothed Binary Value (Brightness)", smoothed_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

Find/Draw Outlines and Centers

In [109]:
contours, _ = cv2.findContours(smoothed_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [110]:
# Remove noise
filtered_contours = []
for contour in contours:
    area = cv2.contourArea(contour)
    # How much noise to filter
    if area > 500:
        filtered_contours.append(contour)

In [111]:
# Calculate centers of contours
centers = []
for contour in filtered_contours:
    # Calculate moments
    M = cv2.moments(contour)
    if M["m00"] != 0: 
        cx = int(M["m10"] / M["m00"])  # x coord
        cy = int(M["m01"] / M["m00"])  # y coord
        centers.append((cx, cy))

In [112]:
# Copy original image to draw on
result = img.copy()

# Draw outlines of shapes
cv2.drawContours(result, filtered_contours, -1, (0, 255, 0), 2)

# Draw centers and place text
for i, (contour, center) in enumerate(zip(filtered_contours, centers)):
        cx, cy = center
        
        # Get bounding box for text placement
        x, y, w, h = cv2.boundingRect(contour)
        
        # Draw center
        cv2.circle(result, center, 5, (255, 255, 255), -1)
        
        # Place text
        text = f"({cx},{cy})"
        text_x = cx - 80
        text_y = y + h + 40
        cv2.putText(result, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2)

In [113]:
if show_steps:
    # Display Result
    cv2.imshow("Result", result)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

Save Annotated Image to Output Folder

In [114]:
output_dir = os.path.join("output", "brightness")
output_name = f"annotated_{file_name}"
output = os.path.join(output_dir, output_name)

In [115]:
cv2.imwrite(output, result)

True